In [1]:
import os, math
import numpy as np
import xarray as xr
from pathlib import Path

ROOT_DIR = Path("../data").resolve()
DATA_DIR = ROOT_DIR / "run"
def area():
    data_file = DATA_DIR / "diff.nc"
    area_file = DATA_DIR / "Area.nc"

    ds = xr.open_dataset(data_file)

    lat = ds["lat"].values
    lon = ds["lon"].values

    r = 6.37122e6  # m

    dlat = float(np.abs(np.median(np.diff(lat))))
    dlon = float(np.abs(np.median(np.diff(lon))))

    lat1 = np.deg2rad(lat - dlat / 2.0)
    lat2 = np.deg2rad(lat + dlat / 2.0)
    dlon_rad = np.deg2rad(dlon)

    area_lat = (r**2) * dlon_rad * np.abs(np.sin(lat2) - np.sin(lat1)) 

    area2d = area_lat[:, None] * np.ones((1, lon.size), dtype=np.float64)


    total_m2 = area2d.sum()
    print(f"The total area (sum of grid cells): {total_m2:.3e} m^2")
    print(f"The total area: {total_m2/1e12:.3f} million km^2")

    out = xr.Dataset(
        {"area": (("lat", "lon"), area2d)},
        coords={"lat": ds["lat"], "lon": ds["lon"]},
    )
    out.to_netcdf(area_file)
    print("Area completed")

area()

The total area (sum of grid cells): 5.101e+14 m^2
The total area: 510.100 million km^2
Area completed
